In [1]:
!pip install pickle5

     |████████████████████████████████| 133kB 8.1MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219266 sha256=0c339b5eabb096feabe6e06cb3e431875e12baaa2886270b36cc3092b7bf2551
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [2]:
import pickle5
import numpy as np
from sklearn.model_selection import train_test_split

# import the necessary packages
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers.experimental import preprocessing

import matplotlib.pyplot as plt

In [4]:
with open('data.pickle', 'rb') as handle:
    data = pickle5.load(handle)

In [17]:
data.keys()

dict_keys(['X', 'y'])

In [18]:
total_sample = len(data["X"])
print(total_sample)

999


In [5]:
X = np.array(data["X"])
y = np.array(data["y"])

num_elems = X.shape[0]
print(num_elems)
print(X[567])
print(y[567])

999
[265. 576.]
[ 1.11   -0.11    1.0625]


In [23]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=[go.Scatter3d(x=y[:,0], y=y[:,1], z=y[:,2],
                                   mode='markers',
                                      marker=dict(
                                        size=2,
                                        colorscale='Viridis',   # choose a colorscale
                                        opacity=0.8
                                    ))])
fig.show()

# Split dataset

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=128)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.09, shuffle=True, random_state=128)

In [21]:
X_train.shape

(899, 2)

In [22]:
y

array([[ 1.13  ,  0.23  ,  1.0625],
       [ 1.2   ,  0.1   ,  1.0625],
       [ 1.2   , -0.93  ,  1.0625],
       ...,
       [ 1.37  , -0.15  ,  1.0625],
       [ 1.3   , -0.45  ,  1.0625],
       [ 1.04  ,  0.11  ,  1.0625]])

# NN vision model

In [8]:
def create_model():
    model = Sequential()
    model.add(Dense(128, activation="relu"))
    model.add(Dense(3))
    sgd = tf.optimizers.SGD(0.001, momentum = 0.3, nesterov = True)
    adam = tf.optimizers.Adam(learning_rate=0.001)
    model.compile(loss="mean_squared_error", optimizer=adam)
    return model

In [13]:
from tensorflow.keras.callbacks import EarlyStopping, Callback
import numpy as np

class ReturnBestEarlyStopping(EarlyStopping):
    def __init__(self, **kwargs):
        super(ReturnBestEarlyStopping, self).__init__(**kwargs)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            if self.verbose > 0:
                print(f'\nEpoch {self.stopped_epoch + 1}: early stopping')
        elif self.restore_best_weights:
            if self.verbose > 0:
                print('Restoring model weights from the end of the best epoch.')
            self.model.set_weights(self.best_weights)

In [14]:
# train the model using SGD
print("[INFO] training network...")
model = create_model()
best_callback = ReturnBestEarlyStopping(monitor="val_loss", patience=50, verbose=0, mode="min", restore_best_weights=True)
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000, callbacks=[best_callback], verbose = 2)

[INFO] training network...
Epoch 1/1000
26/26 - 0s - loss: 428.2440 - val_loss: 9.4368
Epoch 2/1000
26/26 - 0s - loss: 16.2505 - val_loss: 3.4398
Epoch 3/1000
26/26 - 0s - loss: 1.8493 - val_loss: 1.1224
Epoch 4/1000
26/26 - 0s - loss: 0.9063 - val_loss: 0.7276
Epoch 5/1000
26/26 - 0s - loss: 0.6243 - val_loss: 0.5103
Epoch 6/1000
26/26 - 0s - loss: 0.4926 - val_loss: 0.3906
Epoch 7/1000
26/26 - 0s - loss: 0.4055 - val_loss: 0.3577
Epoch 8/1000
26/26 - 0s - loss: 0.3452 - val_loss: 0.2926
Epoch 9/1000
26/26 - 0s - loss: 0.2977 - val_loss: 0.2733
Epoch 10/1000
26/26 - 0s - loss: 0.2762 - val_loss: 0.2196
Epoch 11/1000
26/26 - 0s - loss: 0.2524 - val_loss: 0.2373
Epoch 12/1000
26/26 - 0s - loss: 0.2294 - val_loss: 0.2125
Epoch 13/1000
26/26 - 0s - loss: 0.2088 - val_loss: 0.1880
Epoch 14/1000
26/26 - 0s - loss: 0.1925 - val_loss: 0.1704
Epoch 15/1000
26/26 - 0s - loss: 0.1834 - val_loss: 0.1652
Epoch 16/1000
26/26 - 0s - loss: 0.1680 - val_loss: 0.1672
Epoch 17/1000
26/26 - 0s - loss: 0.

In [15]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 2ms/step - loss: 0.0039


0.003928530961275101

In [17]:
# Save the weights
model.save_weights('./checkpoints_vision/nn_checkpoint_vision')

In [18]:
!zip -r filename.zip ./checkpoints_vision

  adding: checkpoints_vision/ (stored 0%)
  adding: checkpoints_vision/nn_checkpoint_vision.index (deflated 58%)
  adding: checkpoints_vision/nn_checkpoint_vision.data-00000-of-00001 (deflated 30%)
  adding: checkpoints_vision/checkpoint (deflated 51%)


In [16]:
print(model.predict(X_test[0].reshape(1,2)))
print(y_test[0])

[[1.5316721  0.20004682 1.0472419 ]]
[1.57   0.3    1.0625]
